In [1]:
# Step C0 — Setup & load matrices/labels/pop (idempotent)
# - Reuses your test slice path and artefacts from 02a/02b.

from __future__ import annotations
from pathlib import Path
from typing import Optional, Tuple, Dict, List
import warnings, math

import numpy as np
import pandas as pd
from scipy import sparse

BASE = Path("/Users/rosstaylor/Downloads/Code Repositories/REACH Map (NHS SW)") \
    / "GitHub Repo" / "REACH-Map-NHS-SW" / "data" / "raw" / "test_data_ICB_level"
TABLES   = BASE / "tables"
MATRICES = BASE / "matrices"

RESPONSE_THRESHOLDS    = (7, 15, 18, 40)
SCENE_TO_AE_THRESHOLDS = (30, 45, 60)

def _ok(msg: str) -> None: print(f"[OK] {msg}")
def _warn(msg: str) -> None: warnings.warn(msg, stacklevel=2)
def _die(msg: str) -> None: raise RuntimeError(msg)

# Load matrices or reuse from memory if present
try:
    R
    C
except NameError:
    from scipy import sparse
    R = sparse.load_npz(MATRICES / "R_csr.npz")
    C = sparse.load_npz(MATRICES / "C_csr.npz")

meta = np.load(MATRICES / "matrix_metadata.npz", allow_pickle=True)
lsoa_codes    = meta["lsoa_codes"].astype(str)
station_lsoas = meta["station_lsoas"].astype(str)
acute_lsoas   = meta["acute_lsoas"].astype(str)
lsoa_index = pd.Index(lsoa_codes, name="lsoa_code")
N, J, K = len(lsoa_index), len(station_lsoas), len(acute_lsoas)

pop = (pd.read_parquet(TABLES / "population_by_lsoa.parquet")
       .set_index("lsoa_code")["population"]
       .astype("float32").reindex(lsoa_index).fillna(0.0))

_ok(f"Matrices loaded: R={R.shape}, C={C.shape}; N={N}, J={J}, K={K}")


[OK] Matrices loaded: R=(336, 14), C=(336, 3); N=336, J=14, K=3


In [2]:
# Step C1 — Fast reducers & baseline (reuse if already defined)

def min_response_time(R: sparse.csr_matrix, active_cols: Optional[np.ndarray]=None) -> np.ndarray:
    cols = np.arange(R.shape[1]) if active_cols is None else np.asarray(active_cols)
    if cols.size == 0:
        return np.full(R.shape[0], np.inf, dtype="float32")
    S = R[:, cols].tocsr()
    A = S.toarray().astype("float32")
    A[A == 0] = np.inf
    return A.min(axis=1).astype("float32")

def scenario_eval(
    R: sparse.csr_matrix,
    C: sparse.csr_matrix,
    pop: pd.Series,
    resp_thresholds: Tuple[int,...] = RESPONSE_THRESHOLDS,
    conv_thresholds: Tuple[int,...] = SCENE_TO_AE_THRESHOLDS,
    station_cols: Optional[np.ndarray] = None,
    acute_cols: Optional[np.ndarray] = None,
) -> Dict[str, dict]:
    t_resp = min_response_time(R, station_cols)
    t_conv = min_response_time(C, acute_cols)  # same reducer works
    total_pop = float(pop.sum())
    out = {"resp": {}, "conv": {}, "mean_resp_min": float(np.average(np.nan_to_num(t_resp, nan=np.inf), weights=pop.values))}
    for t in resp_thresholds:
        mask = (t_resp <= t)
        out["resp"][t] = {
            "lsoas_cov": int(mask.sum()),
            "pop_cov": int(pop.values[mask].sum()),
            "pop_pct": round(100.0 * float(pop.values[mask].sum()) / total_pop, 2) if total_pop else 0.0,
        }
    for t in conv_thresholds:
        mask = (t_conv <= t)
        out["conv"][t] = {
            "lsoas_cov": int(mask.sum()),
            "pop_cov": int(pop.values[mask].sum()),
            "pop_pct": round(100.0 * float(pop.values[mask].sum()) / total_pop, 2) if total_pop else 0.0,
        }
    return out

baseline = scenario_eval(R, C, pop)
_ok("Baseline (from all stations/acutes) computed.")


[OK] Baseline (from all stations/acutes) computed.


In [3]:
# Step C2 — MCLP helpers (coverage sets) + greedy solver
# - Build, for each demand i, the set of station columns that meet T minutes.
# - Greedy: iteratively add station that covers the most uncovered weighted pop.

def build_coverage_sets(R: sparse.csr_matrix, T: float) -> List[np.ndarray]:
    M = R.copy().tocsr()
    M.data = np.where(M.data <= T, 1.0, 0.0).astype("float32")
    M.eliminate_zeros()
    # For each i (row), return array of j where time <= T
    indptr, indices = M.indptr, M.indices
    cov_sets = [indices[indptr[i]:indptr[i+1]].copy() for i in range(M.shape[0])]
    return cov_sets

def solve_mclp_greedy(
    R: sparse.csr_matrix,
    pop: pd.Series,
    p: int,
    T: float,
    candidate_cols: Optional[np.ndarray] = None,
    weights: Optional[pd.Series] = None,
) -> dict:
    J = R.shape[1]
    cand = np.arange(J) if candidate_cols is None else np.asarray(candidate_cols)
    cov_sets = build_coverage_sets(R, T)
    w = pop.values.copy() if weights is None else (pop.values * weights.reindex(pop.index).fillna(1.0).values)
    w = w.astype("float64")

    selected: List[int] = []
    covered = np.zeros(R.shape[0], dtype=bool)
    gains = []

    for _ in range(min(p, cand.size)):
        best_j, best_gain = None, -1.0
        for j in cand:
            if j in selected: continue
            # Incremental gain: sum of weights for currently uncovered i that j can cover
            gain = float(w[~covered][
                [any(j in cs for cs in [cov_sets[i]]) for i in np.where(~covered)[0]]
            ].sum()) if covered.any() else float(sum(w[i] for i in range(len(cov_sets)) if j in cov_sets[i]))
            if gain > best_gain:
                best_gain = gain
                best_j = int(j)
        if best_j is None: break
        selected.append(best_j)
        # Update covered set
        js = set(selected)
        for i, js_i in enumerate(cov_sets):
            if not covered[i] and any(j in js for j in js_i):
                covered[i] = True
        gains.append(best_gain)

    # KPIs after selection
    sel = np.array(selected, dtype=int)
    kpis = scenario_eval(R, C, pop, station_cols=sel, acute_cols=np.arange(C.shape[1]))
    return {
        "selected_cols": sel,
        "selected_lsoas": station_lsoas[sel],
        "gains": gains,
        "kpis": kpis,
    }


In [4]:
# Step C3 — MCLP MILP (PuLP, optional). Falls back to greedy if PuLP missing.

def solve_mclp_pulp(
    R: sparse.csr_matrix,
    pop: pd.Series,
    p: int,
    T: float,
    candidate_cols: Optional[np.ndarray] = None,
    weights: Optional[pd.Series] = None,
) -> dict:
    try:
        import pulp
    except Exception:
        _warn("PuLP not available; using greedy MCLP instead.")
        return solve_mclp_greedy(R, pop, p, T, candidate_cols, weights)

    cand = np.arange(R.shape[1]) if candidate_cols is None else np.asarray(candidate_cols)
    cov_sets = build_coverage_sets(R, T)
    w = pop.values.copy() if weights is None else (pop.values * weights.reindex(pop.index).fillna(1.0).values)
    w = w.astype("float64")

    prob = pulp.LpProblem("MCLP", pulp.LpMaximize)
    x = pulp.LpVariable.dicts("x", list(cand), lowBound=0, upBound=1, cat=pulp.LpBinary)
    y = pulp.LpVariable.dicts("y", list(range(R.shape[0])), lowBound=0, upBound=1, cat=pulp.LpContinuous)

    # Coverage constraints: y_i ≤ Σ_j∈cover(i) x_j
    for i, js in enumerate(cov_sets):
        in_cand = [int(j) for j in js if j in set(cand)]
        if in_cand:
            prob += (y[i] <= pulp.lpSum(x[j] for j in in_cand))
        else:
            prob += (y[i] <= 0)

    # Cardinality: Σ x_j = p
    prob += pulp.lpSum(x[j] for j in cand) == p

    # Objective
    prob += pulp.lpSum(w[i] * y[i] for i in range(R.shape[0]))

    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    selected = np.array([j for j in cand if pulp.value(x[j]) > 0.5], dtype=int)

    kpis = scenario_eval(R, C, pop, station_cols=selected, acute_cols=np.arange(C.shape[1]))
    return {
        "selected_cols": selected,
        "selected_lsoas": station_lsoas[selected],
        "solver": "PuLP/CBC",
        "kpis": kpis,
    }


In [5]:
# Step C4 — p-median (PuLP with k-nearest), greedy fallback
# - Build k-nearest arcs per demand to keep model small (k ~ 10–12 good here).

def _knn_arcs(R: sparse.csr_matrix, k: int = 12) -> List[Tuple[np.ndarray, np.ndarray]]:
    # Returns per i: (j_idx, t_ij) sorted by time, truncated at k and finite
    out = []
    A = R.tocsr()
    for i in range(A.shape[0]):
        row = A.getrow(i)
        js, ts = row.indices, row.data
        if js.size == 0:
            out.append((np.array([], dtype=int), np.array([], dtype="float32")))
            continue
        order = np.argsort(ts)
        js_sorted = js[order][:k]
        ts_sorted = ts[order][:k].astype("float32")
        out.append((js_sorted, ts_sorted))
    return out

def solve_pmedian_pulp(
    R: sparse.csr_matrix,
    pop: pd.Series,
    p: int,
    k_nearest: int = 12,
    weights: Optional[pd.Series] = None,
) -> dict:
    try:
        import pulp
    except Exception:
        _warn("PuLP not available; using greedy p-median instead.")
        return solve_pmedian_greedy(R, pop, p, k_nearest, weights)

    knn = _knn_arcs(R, k=k_nearest)
    w = pop.values.copy() if weights is None else (pop.values * weights.reindex(pop.index).fillna(1.0).values)
    w = w.astype("float64")

    prob = pulp.LpProblem("p_median", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", list(range(R.shape[1])), lowBound=0, upBound=1, cat=pulp.LpBinary)
    z = {}  # assignment vars only on knn arcs

    # Create z_ij only where knn arc exists
    for i, (js, ts) in enumerate(knn):
        for j in js:
            z[(i, int(j))] = pulp.LpVariable(f"z_{i}_{int(j)}", lowBound=0, upBound=1, cat=pulp.LpContinuous)

    # Each demand assigned to exactly one open facility among its knn
    for i, (js, _) in enumerate(knn):
        if js.size > 0:
            prob += pulp.lpSum(z[(i, int(j))] for j in js) == 1
        else:
            # No arcs: skip (or could add big-M to a dummy facility)
            pass

    # Cannot assign to closed facilities
    for (i, j), var in z.items():
        prob += var <= x[j]

    # Exactly p facilities
    prob += pulp.lpSum(x[j] for j in range(R.shape[1])) == p

    # Objective: Σ_i Σ_j w_i * t_ij * z_ij
    obj_terms = []
    for i, (js, ts) in enumerate(knn):
        for j, t in zip(js, ts):
            obj_terms.append(w[i] * float(t) * z[(i, int(j))])
    prob += pulp.lpSum(obj_terms)

    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    selected = np.array([j for j in range(R.shape[1]) if pulp.value(x[j]) > 0.5], dtype=int)

    # KPIs
    t_resp_sel = min_response_time(R, selected)
    mean_time = float(np.average(t_resp_sel, weights=pop.values))
    kpis = scenario_eval(R, C, pop, station_cols=selected, acute_cols=np.arange(C.shape[1]))
    kpis["mean_resp_min"] = mean_time

    return {
        "selected_cols": selected,
        "selected_lsoas": station_lsoas[selected],
        "solver": "PuLP/CBC (k-nearest)",
        "kpis": kpis,
    }

def solve_pmedian_greedy(
    R: sparse.csr_matrix,
    pop: pd.Series,
    p: int,
    k_nearest: int = 12,
    weights: Optional[pd.Series] = None,
) -> dict:
    # Simple forward-selection: add the station that most reduces weighted mean time
    w = pop.values.copy() if weights is None else (pop.values * weights.reindex(pop.index).fillna(1.0).values)
    w = w.astype("float64")
    selected: List[int] = []
    current = np.full(R.shape[0], np.inf, dtype="float32")

    for _ in range(min(p, R.shape[1])):
        best_j, best_obj = None, math.inf
        for j in range(R.shape[1]):
            if j in selected: continue
            trial = np.minimum(current, R[:, j].toarray().ravel().astype("float32"))
            trial_wavg = float(np.average(np.nan_to_num(trial, nan=np.inf), weights=w))
            if trial_wavg < best_obj:
                best_obj, best_j = trial_wavg, j
        if best_j is None: break
        selected.append(best_j)
        current = np.minimum(current, R[:, best_j].toarray().ravel().astype("float32"))

    kpis = scenario_eval(R, C, pop, station_cols=np.array(selected), acute_cols=np.arange(C.shape[1]))
    kpis["mean_resp_min"] = float(np.average(np.nan_to_num(current, nan=np.inf), weights=w))
    return {
        "selected_cols": np.array(selected, dtype=int),
        "selected_lsoas": station_lsoas[np.array(selected, dtype=int)],
        "solver": "greedy",
        "kpis": kpis,
    }


In [6]:
# Step C5 — Run example experiments + save CSVs
# - Tune p and T as needed (e.g., p=10, T=15).
# - Outputs: CSVs with chosen station LSOAs and KPI deltas.

p = min(10, J)     # example: pick up to 10 stations
T = 15             # MCLP threshold (minutes)

# MCLP
mclp = solve_mclp_pulp(R, pop, p=p, T=T)  # falls back to greedy if PuLP not available
sel_mclp = mclp["selected_cols"]
kpis_mclp = mclp["kpis"]

# p-median
pmed = solve_pmedian_pulp(R, pop, p=p, k_nearest=12)  # falls back to greedy
sel_pmed = pmed["selected_cols"]
kpis_pmed = pmed["kpis"]

# Baseline KPIs for delta
base = scenario_eval(R, C, pop)

def _kpi_line(title, kpis):
    return f"{title} | " + " | ".join(f"T{t}:{d['lsoas_cov']} ({d['pop_pct']}%)"
                                      for t, d in kpis["resp"].items()) \
           + f" | mean_resp={kpis.get('mean_resp_min', float('nan')):.2f} min"

print(_kpi_line("[BASE]", base))
print(_kpi_line("[MCLP]", kpis_mclp))
print(_kpi_line("[PMED]", kpis_pmed))

# Save selections
mclp_df = pd.DataFrame({
    "station_col": sel_mclp,
    "station_lsoa": station_lsoas[sel_mclp],
})
pmed_df = pd.DataFrame({
    "station_col": sel_pmed,
    "station_lsoa": station_lsoas[sel_pmed],
})

mclp_df.to_csv(MATRICES / f"mclp_solution_p{p}_T{T}.csv", index=False)
pmed_df.to_csv(MATRICES / f"pmedian_solution_p{p}.csv", index=False)
_ok(f"Wrote mclp_solution_p{p}_T{T}.csv and pmedian_solution_p{p}.csv")


[BASE] | T7:106 (30.75%) | T15:213 (62.28%) | T18:253 (75.11%) | T40:334 (99.26%) | mean_resp=13.04 min
[MCLP] | T7:97 (28.39%) | T15:193 (56.56%) | T18:233 (69.39%) | T40:334 (99.26%) | mean_resp=14.17 min
[PMED] | T7:97 (28.39%) | T15:193 (56.56%) | T18:233 (69.39%) | T40:334 (99.26%) | mean_resp=14.17 min
[OK] Wrote mclp_solution_p10_T15.csv and pmedian_solution_p10.csv


In [7]:
# Step C6 — Equity weights hook (optional)
# - Example: IMD Q1 = 1.5; others = 1.0

try:
    equity = pd.read_parquet(TABLES / "lsoa_lookup_equity.parquet").set_index("lsoa_code")
    w = pd.Series(1.0, index=lsoa_index, dtype="float32")
    if "imd_quintile" in equity.columns:
        w.loc[equity["imd_quintile"] == 1] = 1.5
        _ok("Equity weights set: IMD Q1 boosted to 1.5x.")
        # Re-run a weighted MCLP
        mclp_w = solve_mclp_pulp(R, pop, p=p, T=T, weights=w)
        print(_kpi_line("[MCLP + Equity]", mclp_w["kpis"]))
except Exception as e:
    _warn(f"Equity weighting skipped: {e}")


[OK] Equity weights set: IMD Q1 boosted to 1.5x.
[MCLP + Equity] | T7:97 (28.39%) | T15:193 (56.56%) | T18:233 (69.39%) | T40:334 (99.26%) | mean_resp=14.17 min
